# Création de prompts pour le nommage et la description des clusters

In [ ]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_kmeans.rds") # nolint
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_2025.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_composition.rds")

In [ ]:
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

prompts <- list()

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculer quelques statistiques démographiques pour le cluster
  perc_hommes     <- mean(cluster_data$ses_genderMale, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$ses_immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$ses_languageEnglish, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$ses_languageFrench, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethnicityWhite, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_sexOrientationHetero, na.rm = TRUE) * 100
  educ_mean_BHS       <- mean(cluster_data$ses_educBHS, na.rm = TRUE) * 100
  educ_mean_PostHS       <- mean(cluster_data$ses_educPostHS, na.rm = TRUE) * 100
  educ_mean_Univ       <- mean(cluster_data$ses_educUniv, na.rm = TRUE) * 100
  age_mean        <- mean(cluster_data$ses_age, na.rm = TRUE) * 80
  revenu_moyen_Low    <- mean(cluster_data$ses_incomeLow, na.rm = TRUE) * 100
  revenu_moyen_Mid    <- mean(cluster_data$ses_incomeMid, na.rm = TRUE) * 100
  revenu_moyen_High    <- mean(cluster_data$ses_incomeHigh, na.rm = TRUE) * 100

  # Génération du prompt de suggestion de noms
  prompt_description <- (paste0(
  "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona ", cluster_value, " :\n\n",
  
  "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "green")) {
    paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
  } else {
    "Aucune variable n'a une moyenne significativement plus élevée que la moyenne de la population en général."
  },
  
  "\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "red")) {
    paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
  } else {
    "Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général."
  },
  
  "\n\nCe cluster est composé de :\n", 
  "Hommes: ", round(perc_hommes, 1), "%, ",
  "Immigrants: ", round(perc_immigrants, 1), "%\n",
  "Âge: ", round(age_mean, 1), " ans\n",
  "Revenus:\n",
  "  Revenu Bas: ", round(revenu_moyen_Low, 1), " %\n",
  "  Revenu Mid: ", round(revenu_moyen_Mid, 1), " %\n",
  "  Revenu High: ", round(revenu_moyen_High, 1), " %\n",
  "Langues:\n",
  "  En: ", round(perc_langEn, 1), "%, ",
  "Fr: ", round(perc_langFr, 1), "%, ",
  "Autres: ", round(perc_ses_languageOther, 1), "%\n",
  "EducBHS: ", round(educ_mean_BHS, 1), "%\n",
  "EducPostHS: ", round(educ_mean_PostHS, 1), "%\n",
  "EducUniv: ", round(educ_mean_Univ, 1), "%\n",
  "Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
  "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n"
))

  prompts[[cluster_value]] <- prompt_description
}


In [ ]:
prompts

In [ ]:
reponses_openai <- list()
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  reponses_openai[[cluster]] <- resp$content
}


In [ ]:
reponses_openai

In [ ]:
description_clusters <- data.frame(prompt = unlist(prompts), description = unlist(reponses_openai))

In [ ]:
description_clusters

In [ ]:
write.csv(description_clusters, "description_clusters.csv", row.names = FALSE)

In [ ]:
saveRDS(description_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_description.rds")